# Pneumonia Detection from Chest X-Ray Images using CNN

## Step 1 : Importing the libraries

In [42]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

In [43]:
tf.__version__

'2.18.0'

## Step 2 : Load and Preprocess the Data

In [44]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [45]:
train_dir = '/content/drive/MyDrive/chest_xray/train'
val_dir = '/content/drive/MyDrive/chest_xray/val'
test_dir = '/content/drive/MyDrive/chest_xray/test'


## Step 3 : Create Image Generators

In [46]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    shear_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')
val_test_datagen = ImageDataGenerator(rescale=1./255)

## Step 4 : Flow the images from directories

In [47]:
train_generator = train_datagen.flow_from_directory(train_dir, target_size=(64, 64), batch_size=32, class_mode='binary')
val_generator = val_test_datagen.flow_from_directory(val_dir, target_size=(64,64), batch_size=32, class_mode='binary')
test_generator = val_test_datagen.flow_from_directory(test_dir, target_size=(64, 64), batch_size=32, class_mode='binary', shuffle=False)

Found 5236 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 544 images belonging to 2 classes.


## Step 5 : Build the CNN Model

In [48]:
from tensorflow.keras import layers, models


In [49]:
from tensorflow.keras import layers, models

model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(64,64,3)),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),  # Dropout added here
    layers.Dense(1, activation='sigmoid')
])


## Step 6 : Compile the Model

In [50]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


## Step 7 : Add Callbacks (Early Stopping)

In [51]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

callback = [
    EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2)
]



## Step 8 : Train the Model

In [52]:
history = model.fit(train_generator, epochs=10, validation_data=val_generator, callbacks=[callback])


Epoch 1/10
164/164 ━━━━━━━━━━━━━━━━━━━━ 68s 393ms/step - accuracy: 0.7475 - loss: 0.5186 - val_accuracy: 0.8750 - val_loss: 0.4881 - learning_rate: 0.0010
Epoch 2/10
164/164 ━━━━━━━━━━━━━━━━━━━━ 78s 381ms/step - accuracy: 0.8805 - loss: 0.2793 - val_accuracy: 0.6875 - val_loss: 0.9569 - learning_rate: 0.0010
Epoch 3/10
164/164 ━━━━━━━━━━━━━━━━━━━━ 62s 376ms/step - accuracy: 0.9067 - loss: 0.2289 - val_accuracy: 0.6875 - val_loss: 0.6409 - learning_rate: 0.0010
Epoch 4/10
164/164 ━━━━━━━━━━━━━━━━━━━━ 83s 380ms/step - accuracy: 0.9283 - loss: 0.1959 - val_accuracy: 0.6875 - val_loss: 0.5266 - learning_rate: 2.0000e-04


## Step 9 : Evaluate the Model

In [53]:
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc:.2f}")


17/17 ━━━━━━━━━━━━━━━━━━━━ 6s 322ms/step - accuracy: 0.8779 - loss: 0.3948
Test Accuracy: 0.86


## Step 10 : Confusion Matrix

In [54]:
from sklearn.metrics import confusion_matrix, classification_report , accuracy_score

y_pred = model.predict(test_generator)
y_pred_classes = (y_pred > 0.5).astype(int)
y_true = test_generator.classes

print(confusion_matrix(y_true, y_pred_classes))
print(classification_report(y_true, y_pred_classes))

print(accuracy_score(y_true, y_pred_classes))

17/17 ━━━━━━━━━━━━━━━━━━━━ 6s 277ms/step
[[201  33]
 [ 44 266]]
              precision    recall  f1-score   support

           0       0.82      0.86      0.84       234
           1       0.89      0.86      0.87       310

    accuracy                           0.86       544
   macro avg       0.86      0.86      0.86       544
weighted avg       0.86      0.86      0.86       544

0.8584558823529411
